In [56]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, TimeoutException

from bs4 import BeautifulSoup

import pandas as pd
import time
import pyautogui
import requests

In [58]:
url = "https://www.google.com/maps/place/Miacucina%E4%BF%A1%E7%BE%A9%E5%BA%97/data=!4m7!3m6!1s0x3442abba740a1a33:0xe886c8c9afd05088!8m2!3d25.0361121!4d121.5672692!16s%2Fg%2F11cly3t3k0!19sChIJMxoKdLqrQjQRiFDQr8nIhug?authuser=0&hl=zh-TW&rclk=1"

driver = webdriver.Chrome()
driver.get(url)
wait = WebDriverWait(driver,10)

"""
商家資訊網頁評論區按鍵路徑:
    //*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div/div/button[2]/div[2]/div[2]
    
評論內容篩選按鍵路徑:
    //*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[7]/div[2]/button

評論內容篩選最新留言排序按鍵路徑:
    //*[@id="action-menu"]/div[2]
    
抓取留言時間，每則路徑不一樣所以用class name定位
    <span class="rsqaWe">4 天前</span>

抓取留言內容，每則路徑不一樣所以用class name定位
    <span class="wiI7pd">新天地開的店, 高檔火煱, 食材, 服務沒得挑惕, 海陸都完美, 宴客最佳選擇.</span>
    
展開留言全文內容class name 
    <button class="w8nwRe kyuRq" aria-expanded="false" jsaction="pane.review.expandReview" aria-controls="ChdDSUhNMG9nS0VJQ0FnSURwLWVmem9RRRAB" data-review-id="ChdDSUhNMG9nS0VJQ0FnSURwLWVmem9RRRAB" jslog="63707; track:click;metadata:WyIwYWhVS0V3aUsycTdWcm9PQkF4V2IzUUlISFU1OUQ0b1E0UjRJUHlnRSJd" aria-label="顯示更多">全文</button>
"""

comments_key_xpath = "//*[@id=\"QA0Szd\"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div/div/button[2]/div[2]/div[2]"
dropdown_key_xpath = "//*[@id=\"QA0Szd\"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[7]/div[2]/button"
newest_comment_key_xpath = "//*[@id=\"action-menu\"]/div[2]"
comment_content_class_name = "wiI7pd" # 沒用
comment_post_time_class_name = "rsqaWe" 
expand_comment_content_class_name = "w8nwRe kyuRq" # 沒用

comments_key = wait.until(expected_conditions.presence_of_element_located((By.XPATH, comments_key_xpath)))
comments_key.click()
dropdown_key = wait.until(expected_conditions.presence_of_element_located((By.XPATH, dropdown_key_xpath)))
dropdown_key.click()
newest_comments_key = wait.until(expected_conditions.presence_of_element_located((By.XPATH,newest_comment_key_xpath)))
newest_comments_key.click()

# 透過滑動網頁加載留言資料，大致確保能加載到近3個月的留言資料，正式爬取設定為200次
scroll_times = 100
for _ in range(scroll_times):
    pyautogui.press('pagedown')
    time.sleep(1)

"""
每個評論留言區塊中若有打開全文按鍵中網頁代碼比較
css_selector 不一樣
JS Path 一樣
XPath 不一樣
full XPath 些微不一樣

餐廳資訊評論區頁面評論中美評論留言中打開全文按鍵xpath可能組合
    /html/body/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[9]/div[1]/div/div/div[4]/div[2]/div/span[2]/button
    /html/body/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[9]/div[16]/div/div/div[4]/div[2]/div/span[2]/button
    /html/body/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[9]/div[1]/div/div/div[4]/div[2]/div/span[2]/button
    /html/body/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[9]/div[4]/div/div/div[4]/div[2]/div/span[2]/button
"""

expand_comment_content_key_xpath = "/html/body/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[9]/div[num]/div/div/div[4]/div[2]/div/span[2]/button"
expand_comment_content_key_xpath_list = []
for i in range(0, 9999):
    expand_comment_content_key_generate_xpath = expand_comment_content_key_xpath.replace('[num]', f'[{i}]')
    expand_comment_content_key_xpath_list.append(expand_comment_content_key_generate_xpath)

for ele in expand_comment_content_key_xpath_list:
    try:
        expand_button = driver.find_element(By.XPATH,ele)
        expand_button.click()
        # print(f"已打開評論全文按鍵 {ele}")
    except NoSuchElementException:
        print(f"找不到對應打開全文按鍵的xpath:{ele}")
        # pass
    

找不到對應打開全文按鍵的xpath:/html/body/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[9]/div[0]/div/div/div[4]/div[2]/div/span[2]/button
找不到對應打開全文按鍵的xpath:/html/body/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[9]/div[2]/div/div/div[4]/div[2]/div/span[2]/button
找不到對應打開全文按鍵的xpath:/html/body/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[9]/div[3]/div/div/div[4]/div[2]/div/span[2]/button
找不到對應打開全文按鍵的xpath:/html/body/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[9]/div[5]/div/div/div[4]/div[2]/div/span[2]/button
找不到對應打開全文按鍵的xpath:/html/body/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[9]/div[6]/div/div/div[4]/div[2]/div/span[2]/button
找不到對應打開全文按鍵的xpath:/html/body/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[9]/div[8]/div/div/div[4]/div[2]/div/span[2]/button
找不到對應打開全文按鍵的xpath:/html/body/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]

In [59]:
"""
抓取評論留言，若用class name 來定位，抓取到的內容會包含業主回應，應該改用xpath
訪客評論留言內容xpath路徑可能組合
    /html/body/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[9]/div[349]/div/div/div[4]/div[2]/div/span[1]
    /html/body/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[9]/div[352]/div/div/div[4]/div[2]/div/span[1]
"""

comment_content_xpath = "/html/body/div[3]/div[8]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[9]/div[num]/div/div/div[4]/div[2]/div/span[1]"
comment_content_xpath_list = []
for i in range(0,9999):
    comment_content_xpath_generate = comment_content_xpath.replace('[num]',f'[{i}]')
    comment_content_xpath_list.append(comment_content_xpath_generate)

comment_list = []
for xpath in comment_content_xpath_list:
    try:
        comment = driver.find_element(By.XPATH,xpath).text
        comment_list.append(comment)
    except NoSuchElementException:
        # print(f"找不到對應XPath:{xpath}的評論留言")
        pass

# print(comment_list)

# comment_post_time_list = []
# comment_post_time = wait.until(expected_conditions.presence_of_all_elements_located((By.CLASS_NAME,comment_post_time_class_name)))
# for ele in comment_post_time:
#     post_time = ele.text
#     comment_post_time_list.append(post_time)
# print(comment_post_time_list)

In [60]:
print(comment_list)

['超級蛋白質佛陀碗：泰式花生醬好吃，鷹嘴豆好吃辣辣的，地瓜好吃，生菜也不錯，飯有點多餘。\n橙橘草莓胡桃沙拉(小)：水果原味好吃，放挺多蔓越莓的，太甜，但搭配的醋不好吃。\n\n之所以說服務不好，是因為餐具是髒的，有點噁，摸起來油油的，還有一支叉子上有殘渣。\n\n店內很吵，可能因為靠近空橋會有人進出。', '很久沒有去吃了，因為服務態度不佳。今天去用餐，整個讓人改觀，除了裝潢更新，外場工作人員服務品質整體大提升，讓人耳目一新', '吃不出來是素', '餐點很有特色，以蔬食餐廳來說，非常好吃\n服務人員態度親切，出餐速度快速\n可以多人一起分食，可以吃到比較多樣', '餐點具有特色，服務人員的態度很好。', '美味蔬食餐廳，沙拉大份但看起來只是碗變大，內容物還沒。', '在次造訪，餐點一樣美味，服務人員態度親切，跟之前服務態度大有進步👍', '8/10晚餐老婆約會趣~\n對呀~又去了！\n哈~肚子太餓，\n這次忘了拍正餐照....\n\n可惜....\n因沒好的位子，且人太多啦！\n（有機會還是提早預約）\n\n分享：\n7/7.2/21下午茶時光~', '大蒜香菇麵咬下去每一口都充滿著香氣，還有一種類似粽葉飄香的味道，很特殊', 'MiaCucina (My Kitchen) 信義店\n今天來信義區逛街\n不小心看這家餐廳好像很好吃\n本來只是想吃甜點\n進去才發現是素食餐廳\n一不小心點了：\n🍴大壞蛋揍扁酪梨堡（395）\n友人點的，我沒吃無法形容\n但附餐薯條超好吃，番茄醬也感覺是自己調的\n\n🍴青醬蔬菜羊奶酪薄餅（445）\n超討厭彩椒的我居然吃光一大片\n完全沒有青椒的怪味，整體融合的非常好\n羊奶酪有淡淡的羊奶香\n薄餅超薄超脆\n\n🍴蒜油野菇麵（455）\n因為我不吃菇類所以只淺嚐一口麵\n那個橄欖油超香\n竹炭麵煮的有一點點麵芯\n是義大利的味道\n\n🍴焦糖堅果鬆餅（295）\n本日第一名給這個\n鬆餅外脆內軟，目前吃到最好吃的不誇張\n腰果跟核桃不手軟給一堆\n再來那個焦糖醬不知道為什麼有楓糖香氣\n整個愛上😍😍😍😍😍😍\n為了鬆餅可以再去一次沒問題\n\n🍴Oreo 奶昔（195）\n友人的，偷喝一口好好喝，滿滿Oreo味卻不會太甜膩\n\n#Dreamerscoffee集團\n#甜點看起來都好厲害\n#mia我的\n#cucina廚房

In [39]:
print(len(comment_list))

121
